# Story Finder & Generator - Part 2
## Generator and Speech Synthethizer
---
**IRS Innovative Assignment (Even 2023-24)** <br>
**Roll No. and Names:**<br>
&emsp;21BCE183 Parv Thacker <br>
&emsp;21BCE201 Kaju Patel <br>
&emsp;21BCE250 Tanvi Rathod <br>

# Required Installation

In [1]:
!pip install playsound
!pip install gtts
!pip install pyttsx3

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=3eff175dd245d2ea70848ccfd57fad6af0ba301a726722b50b6776a5ac0d6809
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


# Import Libraries

In [2]:
# Getting Data
import ast

# LLM
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

#TTS
import gtts  
from playsound import playsound

# Hide LLM usage warnings
import warnings
warnings.filterwarnings('ignore')

## Load Model

In [3]:
if True:
    print("Initializing. Loading Model.")
    model = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"

    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForCausalLM.from_pretrained(model)
    print("Initializing. Model Loaded.")

Initializing. Loading Model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Initializing. Model Loaded.


# Modules

## Get Data (from part 1)

In [4]:
import ast

class Book:
    def __init__(self, dictionary_str=None, book_data=None):
        # Dictionary Exists
        if book_data:
            self.book_data = book_data
        
        # Dictionary Passed
        elif dictionary_str:
            try:
                dictionary = ast.literal_eval(dictionary_str)
                if not isinstance(dictionary, dict):
                    raise ValueError("Input is not a valid dictionary.")
                self.book_data = dictionary
            except ValueError as e:
                print("Error:", e)
                self.book_data = {}
        
        # Get dictionary from user
        else:
            dictionary_str = input("Enter the dictionary in string format: ")
            try:
                dictionary = ast.literal_eval(dictionary_str)
                if not isinstance(dictionary, dict):
                    raise ValueError("Input is not a valid dictionary.")
                self.book_data = dictionary
            except ValueError as e:
                print("Error:", e)
                self.book_data = {}

    def get_attribute(self, attribute):
        try:
            data = self.book_data[attribute]
        except ValueError as e:
            print("Error:", e)
            self.book_data = {}
        return data
    
    def show_attribute(self, attribute):
        print(f"{attribute:<20}:", self.get_attribute(attribute))
    
    def get_full_data(self):
        return self.book_data

## Llama Language Model

In [5]:
system_prompt = "You are an expert story writer that can write a completely different yet similar story from an already existing book's abstract given as prompt. Ensure that the story doesn't resemble the abstract. Change the names of the places and people. Do not reprint the input or the prompts. Output only the title and the story and nothing else."

def create_pipeline(model, tokenizer):
    print("Initializing. Creating Pipeline.")
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        torch_dtype=torch.float16,
        device_map="auto",
        tokenizer=tokenizer,
    )
    print("Initializing. Pipeline Created.")
    return pipeline


In [11]:
def get_response(pipeline, system_prompt, bookObj):
    prompt = f"<<SYS>>{system_prompt}<</SYS>>\n{bookObj.get_attribute('description')}"
    print("Generating...")

    sequences = pipeline(
        prompt,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=2000,
    )
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")
    
    return seq

## TTS

In [12]:
def do_tts(generated_text = "The Disappearance of Emilia MartinezEmilia Martinez, a young heiress of a wealthy Mexican family, vanished without a trace over three decades ago. Her elderly uncle, a former diplomat, has been obsessed with solving the mystery of her disappearance ever since. Desperate for answers, he hires a skilled journalist, Rodrigo Sanchez, to investigate. Rodrigo is aided by the enigmatic and rebellious aspiring hacker, Sofia Delgado. Together, they delve into a complex web of deceit and uncover a shocking conspiracy that threatens to expose the dark secrets of the Martinez family and their powerful connections."):
    t1 = gtts.gTTS(generated_text)
    t1.save("welcome.mp3")

# Driver Code (Part 2)

In [13]:
book = Book("{'title': 'Swerve',  'rating': 3.54,  'ratings_count': 1506,  'reviews_count': 428,  'description': 'It’s high summer in the Mojave Desert, and Kristine Rush and her fiancé, Daniel, are en route from Las Vegas to Lake Arrowhead, California, for the July Fourth holiday weekend. But when Daniel is abducted from a desolate rest stop, Kristine is forced to choose: return home unharmed, but never to see her fiancé again, or plunge forward into the searing desert to find him…where a killer lies in wait.'}")

print("-"*50)
print("You have Selected:")
print(book.get_full_data())

--------------------------------------------------
You have Selected:
{'title': 'Swerve', 'rating': 3.54, 'ratings_count': 1506, 'reviews_count': 428, 'description': 'It’s high summer in the Mojave Desert, and Kristine Rush and her fiancé, Daniel, are en route from Las Vegas to Lake Arrowhead, California, for the July Fourth holiday weekend. But when Daniel is abducted from a desolate rest stop, Kristine is forced to choose: return home unharmed, but never to see her fiancé again, or plunge forward into the searing desert to find him…where a killer lies in wait.'}


In [14]:
pipeline = create_pipeline(model, tokenizer)

Initializing. Creating Pipeline.
Initializing. Pipeline Created.


In [15]:
generated_text = get_response(pipeline, system_prompt, book)

Generating...
Result: <<SYS>>You are an expert story writer that can write a completely different yet similar story from an already existing book's abstract given as prompt. Ensure that the story doesn't resemble the abstract. Change the names of the places and people. Do not reprint the input or the prompts. Output only the title and the story and nothing else.<</SYS>>
It’s high summer in the Mojave Desert, and Kristine Rush and her fiancé, Daniel, are en route from Las Vegas to Lake Arrowhead, California, for the July Fourth holiday weekend. But when Daniel is abducted from a desolate rest stop, Kristine is forced to choose: return home unharmed, but never to see her fiancé again, or plunge forward into the searing desert to find him…where a killer lies in wait. 
As Kristine navigates through the unforgiving landscape, she realizes that Daniel’s abduction is part of a larger scheme to steal a priceless artifact hidden in a secret cave deep within the desert. With time running out and

In [18]:
generated_text_ = generated_text['generated_text']
generated_text_

'<<SYS>>You are an expert story writer that can write a completely different yet similar story from an already existing book\'s abstract given as prompt. Ensure that the story doesn\'t resemble the abstract. Change the names of the places and people. Do not reprint the input or the prompts. Output only the title and the story and nothing else.<</SYS>>\nIt’s high summer in the Mojave Desert, and Kristine Rush and her fiancé, Daniel, are en route from Las Vegas to Lake Arrowhead, California, for the July Fourth holiday weekend. But when Daniel is abducted from a desolate rest stop, Kristine is forced to choose: return home unharmed, but never to see her fiancé again, or plunge forward into the searing desert to find him…where a killer lies in wait. \nAs Kristine navigates through the unforgiving landscape, she realizes that Daniel’s abduction is part of a larger scheme to steal a priceless artifact hidden in a secret cave deep within the desert. With time running out and danger lurking a

In [22]:
do_tts(generated_text)

In [ ]:
playsound("welcome.mp3")

In [ ]:
print("hi")